<a href="https://colab.research.google.com/github/GoogTech/langchain-tutorials/blob/master/LangGraph_Glossary/LangGraph_Glossary.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Refer to the langgraph tutorials of `v0.2.74` : https://langchain-ai.github.io/langgraph/concepts/low_level/

# Installation

In [3]:
!pip install -U langgraph langchain-openai

In [4]:
!pip show langgraph langchain_openai

Name: langgraph
Version: 0.2.74
Summary: Building stateful, multi-actor applications with LLMs
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-sdk
Required-by: 
---
Name: langchain-openai
Version: 0.3.6
Summary: An integration package connecting OpenAI and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, openai, tiktoken
Required-by: 


# 1.Graphs

In short:
* nodes do the work.
* edges tell what to do next.
* state represents the memory that records what you did.

## 1.1.StateGraph

## 1.2.Compiling your graph

In [ ]:
graph = graph_builder.compile(...)

# 2.State

## 2.1.Schema

### 2.1.1.How to use Pydantic model as graph state

https://langchain-ai.github.io/langgraph/how-tos/state-model/

A `StateGraph` accepts a state_schema argument on initialization `that` specifies the "shape" of the state `that` the nodes in the graph can access and update.

#### Input Validation

In [8]:
from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel

# The overall state of the graph(this is the public state shared across nodes)
class OverallState(BaseModel):
  a: str

def node(state: OverallState):
  return {"a": "goodbye"}

# Build the state graph:
builder = StateGraph(OverallState)
builder.add_node('node', node)
builder.add_edge(START, 'node')
builder.add_edge('node', END)
graph = builder.compile()

# Test the graph with a valid input
graph.invoke({"a": "hello"})

{'a': 'goodbye'}

Invoke the graph with an invalid input

In [9]:
try:
  graph.invoke({"a": 123})
except Exception as e:
    print("An exception was raised because `a` is an integer rather than a string.\n")
    print(e)

An exception was raised because `a` is an integer rather than a string.

1 validation error for OverallState
a
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type


#### Multiple Nodes

In [12]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

from pydantic import BaseModel

# The overall state of the graph (this is the public state shared across nodes)
class OverallState(BaseModel):
    a: str

def bad_node(state: OverallState):
    return { # !!!returns an update to the state!!!
        "a": 123  # Invalid
    }

def ok_node(state: OverallState):
    return {"a": "goodbye"}

# Build the state graph
builder = StateGraph(OverallState)
builder.add_node(bad_node)
builder.add_node(ok_node)
builder.add_edge(START, "bad_node")
builder.add_edge("bad_node", "ok_node")
builder.add_edge("ok_node", END)
graph = builder.compile()

# Test the graph with a valid input
try:
    graph.invoke({"a": "hello"}) # !!!the validation error will occur when ok_node is called!!!
except Exception as e:
    print("An exception was raised because bad_node sets `a` to an integer.")
    print(e)

An exception was raised because bad_node sets `a` to an integer.
1 validation error for OverallState
a
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type


### 2.1.2.How to define input/output schema for your graph

https://langchain-ai.github.io/langgraph/how-tos/input_output_schema/

By default, StateGraph operates with a single schema, and all nodes are expected to communicate using that schema. However, it's also possible to define distinct input and output schemas for a graph.

In [17]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

# Define the schema for the input.
# The input schema ensures that the provided input
# matches the expected structure.
class InputState(TypedDict):
    question: str

# Define the schema for the output.
# The output schema filters the internal data to return
# only the relevant information according to the defined output schema.
class OutputState(TypedDict):
    answer: str

# Define the overall schema, combining both input and output
class OverallState(InputState, OutputState):
    pass

# Define the node that processes the input and generates an answer
def node(state: OverallState):
    return {"question": state["question"], "answer": "This is answer..."}

# Build the graph with input and output schemas specified
builder = StateGraph(OverallState, input=InputState, output=OutputState)
builder.add_node(node)
builder.add_edge(START, "node")
builder.add_edge("node", END)
graph = builder.compile()

# Invoke the graph with and input and print the result.
# Notice that the output of invoke only includes the output schema.
print(graph.invoke({"question": "What is the meaning of life?"}))

{'answer': 'This is answer...'}


### Multiple schemas

#### How to pass private state between nodes

In [29]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

# The overall state of the graph(this is the public state shared across nodes)
class OverallState(TypedDict):
  a: str

# Output from node_1 contains private data that is not part of the overall state
class Node1Output(TypedDict):
  private_data: str

# Node 2 input onlu requests the private data available after node_1
class Node2Input(TypedDict):
  private_data: str

# The private data is only shared between node_1 and node_2
def node_1(state: OverallState) -> Node1Output:
  output = {"private_data": "set by node_1"}
  print(f"Entered node `node_1`: \n\tInput: {state}.\n\tReturn:{output}")
  return output

def node_2(state: Node2Input) -> OverallState:
  output = {"a": "set by node_2"}
  print(f"Entered node `node_2`:\n\tInput: {state}.\n\tReturned: {output}")
  return output

# Node 3 only has access to the overall state (!!!no access to private data from node_1!!!)
def node_3(state: OverallState):
  output = {"a": "set by node_3"}
  print(f"Entered node `node_3`:\n\tInput: {state}.\n\tReturned: {output}")
  return output

# Build the state graph
builder = StateGraph(OverallState)
builder.add_node(node_1) # node_1 is the first node
builder.add_node(node_2) # node_2 is the second node and accepts private data from node_1
builder.add_node(node_3) # node_3 is the third node and does not see the private data
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", "node_3")
builder.add_edge("node_3", END)

graph = builder.compile()

# Invoke the graph with the initial state
response = graph.invoke({"a": "hello"})
print(f"\nOutput of graph invocation: {response}")

Entered node `node_1`: 
	Input: {'a': 'hello'}.
	Return:{'private_data': 'set by node_1'}
Entered node `node_2`:
	Input: {'private_data': 'set by node_1'}.
	Returned: {'a': 'set by node_2'}
Entered node `node_3`:
	Input: {'a': 'set by node_2'}.
	Returned: {'a': 'set by node_3'}

Output of graph invocation: {'a': 'set by node_3'}


# 3.Nodes

# 4.Edges

# 5.Send

# 6.Command

# 7.Persistence

# 8.Threads

# 9.Storage

# 10.Graph Migrations

# 11.Configuration

# 12.interrupt

# 13.Breakpoints

# 14.Subgraphs

# 15.Visualization

# 16.Streaming